#Core concepts pythorch
## Tensors
fundamental structure, like a  number py array but optimice for  math operations
    - creation is done with :torch.tensor,torch.randn, between others
    - you can move them .to(device) or .cuda() 
    - also convert them tu numpy arrays is posible with torch.from_numpy() and .numpy()
in code readding how many dimentions a tensor have is same as how many square parentesis it has at the beguinning of the output when you print it
### broadcasting
Element wise operations are easy to understand,  if you have 2 tensors of lets say 2 dimentions each, and you want to do a  element wise sum you will summ elements on position 0,0  ;0,1; 1,1;1,0  of both tensors so the resuld will be a 2 dimention tensor. Now from this you can  infer  tensors most be same dimention right well not necesary, here is were broadcsting enters the scene.
if one of your tensors is 1 dimension bigger than the other, torch will copy and paste the tensor (one with less dimensions )to fill up the other dimension that is missing.
This concept actually  was born as  a new matematical notation to compress  complex matematical ideas
#### data manipulation
Other operations like slizing or filtering work pretty similar as you will write them in  pyton lists or pandas dataframes
tensor[0,0:2]
tensor[tensor<23]
tensor.reshape(m,n)
tensor.unsqueeze(4) # to add more dimensions
tensor.squeeze(3) to destroy dimensions


In [ ]:
from torch import torch.nn as nn
class textClassifier :
    def __init__(self,vocab_size,embedding_dim,hidden_dim,output_dim):
        super(textClassifier,self).__init__()
        self.embedding=nn.Embedding(vocab_size,embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        # se estan definiendo creo capas ocultas input o embedding layer y output layer basados en el template de las neural networks the pythorch
        #lstm is a rnn that will act as hidden layer to manage secuentiality of text
        def forward(self,text)        :
            """"defines how the conections will be made inside the layers and how data flows"""
            embeded=self.embedding(text)
            outputs,(hidden,cell)=self.lstm(embeded)
            final_output=self.fc(hidden[-1])
            return final_output
vocab_size = 1000  # Tamaño del vocabulario
embedding_dim = 100  # Dimensiones del embedding
hidden_dim = 256  # Dimensiones ocultas
output_dim = 2  # Dimensiones de salida (clasificación binaria)

model = TextClassifier(vocab_size, embedding_dim, hidden_dim, output_dim)    

## Auto grad
 gradients are ecentiall for back propagation and gradient descent like algorithms, so pytorch aoutomatically calculate them
    - to track computations use .requires_grads= True
    - to compute gradients use loss.backward()
    - to desable tracking for eval or inference with torch.no_grad()
    


In [ ]:
class MultiHeadedAttention (nn.Module):
    """this is the heart of transformmers architecture"""
def __init__(delf, h,d_model):
    """inputs:
            h # of heads
            d_model dimentions of each head """    
    super(MultiHeadedAttention).__init__
    self.h=h
    self.d_model=d_model
    self.linear_w0=nn.linear(dmodel,d_model)
def forward (self,query,key,value):
    query=query.view(-1,self.h,int(self.d_model/self.h)).transpose(1,2)
    key=key.view(-1,self.h,int(self.d_model/self.h)).transpose(1,2)
    value=value.vew(-1,self.h,int(self.d_model/self.h)).transpose(1,2)
    z,_=attention(query,key,value)
    z=z.transpose(1,2).contiguos().view(n_palabras,-1)
    z_w0=linnear_w0(z)
    return z_w0
    
    

## trainning loop and loss functions
torch allows you to have more control over  the trainning loop
1. forward pass : outputs= model(inputs)
2. compute loss : loss =criterion (outputs,targets)
3. backward pass : loss.backward()
4. update parameters: optimizer.step()
5. Reset gradients : optimizer.zero_grad()
the nn module has a ton of usable loss function available to uselike
nn.CrossEntropyLoss , nn.MSEloss, nn.BCEloss

In [ ]:
#data creation
import pytorch
import matplotlib.pyplot as plt
X=torch.arrange(0,1,0.025).unsqueeze(1)
Y=2.5*X+1
train_size= int(0.7*len(X))
Xtrain,Ytrain=X[:train_size],Y[:train_size]
Xtest,Ytest=X[train_size:],Y[train_size:]
#definitionof plot functions
def plot(trainData,testData):
    plt.scatter(trainData[0].numpy(), trainData[1].numpy(), label='Entrenamiento', color='green')
    plt.scatter(testData[0].numpy(), testData[1].numpy(), label='Prueba', color='yellow')
    plt.legend()
    plt.show()
#model class
class linearRegModel:
    def __init__(self):
        self.volume= nn.Parameter(torch.randn(1,dtype=torch.float),requires_grad=True)
        self.bias= nn.Parameter(torch.randn(1,dtype=torch.float),requires_grad=True)
    def forward(self,x:torch.Tensor)->torch.Tensor:
        return  self.volume * x+self.bias
        
#model init
torch.manual_seeds(37)# setting random state in sklearn
model=linearRegModel()
model.stateDic()#givesme parameters as a dictionary
#inferences
with torch.inference_mode:
    pred=model(X_test)

## Data sets  and data loaders
These are abstractions that allows  the handeling and processing of big data  for model development in a efficient way

so the dataset class eather saves al the data in a instance or allowws you to acces the data in real time via methods of the class, the dataloader class recives as  a parameter a dataset instance of the class and alows you to shuffle it, separate it in batches, and create  multiple workers to process it in a paralel way.

so for this to work the dataset class needs 
__len__() method that opens a comunication path between data loader and dataset where data loader shares how many samples are contain in the data set class

__get_item__() method that recives the index of the data sample and retrives the data sample itself

each worker having its own subset of indexes retrive the data and put it in a box (when creating the batch) and then  a collate functions convert this individual samples in to a  batched tensor.

### Optimizers
learning rate by the gradient and then substrac the step  to the parameters
##### stochastic GD
it uses a single trinning example wich makes it have to make a lot of steps, and take a d tour before   arriving to a minimun
##### mini batch GD
uses small baches of data to upgrate the   gradient, so it takes less steps, but still we have the problem of the step becoming smaller as it gets to the minimun
##### SGD with momentum
it let the "ball" gain speed as it travels down hill making the  rate larger when its going in the right direction 
##### ADA grad
some times momentum avershut the  minimun so ada fix this alowing a  adaptative learnning rate for each parameter.
#### RMS prop
it keeps the steps to become to small when the gradient is too small but still far from the minimun
###### ADAM
the most  used  in neural networks it takes the mean of the passed gradients so it hass momentumm  enoght to surpass local minimun it calculates a second moment  based on the rood squeare minimun

all this can be found on the sgd module

### transfer learnning and fine tunnin

you can load pre trainned models and  freeze some of its layers
    for params in model.parameters():
        params.requires_grad= false

conecting it to a new layer for classification and train just the  new layer 
import torch.nn as nn
model.fc = nn.Linear(model.fc.in_features, 5)  # e.g., 5 new classes
optimizer = torch.optim.Adam(model.fc.parameters(), lr=1e-3)
#### saving models
 you can sabe the weigths the optimizer state or the full artifact, usualy the weigths  and yo  load them in the same architecture. 
 It's also possible to load intermidient trainning states.


 #### evaluation and inference  
 you can activate eval mode to desabel drop out, disable gradient tracking to save memory and do inference faster
 you can move predictions to cpu if you need it 


Would you like me to create a mini 7-day practice roadmap (with 20–30 min exercises each day) that takes you from tensors → custom model → fine-tuning a pretrained model in PyTorch?